In [2]:
import os
import timm
import logging
import argparse
import pandas as pd
from typing import Optional
from datetime import datetime
import sys
sys.path.append('../')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import torch.utils.data.distributed
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchmetrics import Accuracy, F1Score, Specificity

from pytorch_lightning import LightningModule
from pytorch_lightning.lite import LightningLite
from pytorch_lightning.callbacks import ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.strategies import ParallelStrategy
from pytorch_lightning.utilities.cli import LightningCLI
from pytorch_lightning import Trainer
# from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.plugins import DDPPlugin

from utils.dataset import PapsDetDataset, train_transforms, val_transforms, test_transforms, MAX_IMAGE_SIZE
from utils.collate import collate_fn
#from utils.sampler_by_group import GroupedBatchSampler, create_area_groups
#from utils.losses import SupConLoss, FocalLoss

from models.detection.backbone import PapsBackboneWithFPN
# from vision.torchvision.models.detection.retinanet import RetinaNet
from models.detection.retinanet import PapsRetinaNet
from det_utils.engine import train_one_epoch

In [4]:
backbone = PapsBackboneWithFPN('resnet18', 256)
model = PapsRetinaNet(backbone, num_classes=1, anchor_generator=None)

PapsRetinaNet(
  (backbone): PapsBackboneWithFPN(
    (extra_block): LastLevelP6P7(
      (p6): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (body): FeatureListNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64

In [13]:
import torchvision
backbone = torchvision.models.__dict__['resnet18'](pretrained=False, progress=False)

In [19]:

import torchvision
from torchvision.models.detection.backbone_utils import _resnet_fpn_extractor
# import torchvision.models.detection.backbone_utils as backbone_utils
from torchvision.ops.feature_pyramid_network import LastLevelP6P7
trainable_backbone_layers = 3
backbone = _resnet_fpn_extractor(
    backbone, trainable_backbone_layers, returned_layers=[2, 3, 4], extra_blocks=LastLevelP6P7(256, 256)
)

ImportError: cannot import name '_resnet_fpn_extractor' from 'torchvision.models.detection.backbone_utils' (/home/beomgon/anaconda3/envs/pytorch/lib/python3.7/site-packages/torchvision/models/detection/backbone_utils.py)

In [18]:
from torchvision.models.detection.retinanet import retinanet_resnet50_fpn

In [25]:
from torchvision.models.detection.backbone_utils import BackboneWithFPN, resnet_fpn_backbone, mobilenet_backbone

In [35]:
import  torchvision.models.detection.backbone_utils

In [36]:
torchvision.models.detection.backbone_utils._resnet_fpn_extractor(
    backbone, trainable_backbone_layers, returned_layers=[2, 3, 4], extra_blocks=LastLevelP6P7(256, 256)
)

AttributeError: module 'torchvision.models.detection.backbone_utils' has no attribute '_resnet_fpn_extractor'